# Техническая часть

In [1]:
!pip install -q sentencepiece
import sentencepiece
!pip install -q transformers

!pip install -q comet_ml
import comet_ml
!pip install -q pytorch-lightning

!pip -q install datasets
!pip -q install rouge_score

!git clone -q https://github.com/DanilDmitriev1999/QA

     |████████████████████████████████| 1.2MB 21.1MB/s 
     |████████████████████████████████| 2.1MB 20.8MB/s 
     |████████████████████████████████| 3.3MB 34.2MB/s 
     |████████████████████████████████| 901kB 44.6MB/s 
     |████████████████████████████████| 266kB 19.0MB/s 
     |████████████████████████████████| 522kB 53.7MB/s 
     |████████████████████████████████| 71kB 9.6MB/s 
     |████████████████████████████████| 61kB 7.9MB/s 
     |████████████████████████████████| 808kB 26.8MB/s 
     |████████████████████████████████| 829kB 51.5MB/s 
     |████████████████████████████████| 276kB 52.1MB/s 
     |████████████████████████████████| 112kB 60.1MB/s 
     |████████████████████████████████| 645kB 45.5MB/s 
     |████████████████████████████████| 1.3MB 48.2MB/s 
     |████████████████████████████████| 143kB 51.8MB/s 
     |████████████████████████████████| 296kB 55.6MB/s 
     |████████████████████████████████| 225kB 19.7MB/s 
     |████████████████████████████████| 245kB 21.5MB

In [2]:
import warnings
warnings.filterwarnings('ignore')
import comet_ml

import numpy as np
import collections
import functools
import json
import random
import os
import math
import re

from io import open
from tqdm import tqdm
from pprint import pprint
from typing import List
from sklearn.metrics import accuracy_score
import seaborn as sns
import matplotlib.pyplot as plt

import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
from torch.utils.data import DataLoader

from transformers import (
    AdamW, MT5ForConditionalGeneration,
    T5Tokenizer,
    get_linear_schedule_with_warmup)

from datasets import load_metric

import pytorch_lightning as pl
from pytorch_lightning.loggers import CometLogger
from pytorch_lightning import Trainer, seed_everything

from QA.DataModule.dataset import *
from QA.DataModule.reader import *

from QA.model.BERT import *
from QA.utils.trainer import *

seed_everything(294)
device = 'cuda' if torch.cuda.is_available() else 'cpu'
if device == 'cuda':
    from torch.cuda import LongTensor
else:
    from torch import LongTensor
print(device)

Global seed set to 294


cuda


# Данные

In [3]:
train_file_path = '/content/QA/data/sber_squad/train-v1.1.json'
dev_file_path = '/content/QA/data/sber_squad/dev-v1.1.json'
train = ReadData(train_file_path)
train_data = train.data
dev = ReadData(dev_file_path)
dev_data = dev.data

tokenizer = T5Tokenizer.from_pretrained('google/mt5-small')

In [4]:
train_data[0]

{'context': 'В допетровское время искусство в России имело своим призванием служить исключительно религиозным целям, а так как православная церковь гнушается изваяниями человеческих фигур, то скульптура, в настоящем смысле слова, не могла в древней Руси не только развиваться, но и существовать. Правда, в некоторых местах, в особенности в бывших новгородских областях, пользовались уважением резные и раскрашенные изображения святых, но они были чужды всякого художественного значения и составляли изделия, возникшие под влиянием Запада. Собственно же на Руси проявления пластики ограничивались литьем небольших крестов, образов-складней, выбиванием окладов на образа и резьбою фигурных иконостасов. В числе плодов западно-европейской цивилизации Пётр Великий перенес в него и скульптуру, которая, однако, при этом государе и долго после него находилось здесь в руках приезжих иностранцев. Главным деятелем по части скульптуры в царствование Петра Великого и Анны Иоанновны был К. Б. Растрелли, отец

In [ ]:
tokenizer.special_tokens_map

{'eos_token': '</s>', 'pad_token': '<pad>', 'unk_token': '<unk>'}

In [ ]:
tokenizer('<s>')

{'input_ids': [1042, 263, 669, 1], 'attention_mask': [1, 1, 1, 1]}

In [ ]:
dev_data[0]

{'context': 'В Миссолонги Байрон заболел лихорадкой, продолжая отдавать все свои силы на борьбу за свободу страны. 19 января 1824 года он писал Хэнкопу: Мы готовимся к экспедиции , а 22 января, в день своего рождения, он вошёл в комнату полковника Стенхопа, где было несколько человек гостей, и весело сказал: Вы упрекаете меня, что я не пишу стихов, а вот я только что написал стихотворение . И Байрон прочёл: Сегодня мне исполнилось 36 лет . Постоянно хворавшего Байрона очень тревожила болезнь его дочери Ады. Получив письмо с хорошей вестью о её выздоровлении, он захотел выехать прогуляться с графом Гамба. Во время прогулки пошёл страшный дождь, и Байрон окончательно захворал. Последними словами поэта были отрывочные фразы: Сестра моя! дитя моё!.. бедная Греция!.. я отдал ей время, состояние, здоровье!.. теперь отдаю ей и жизнь! .',
 'id': '2153',
 'qas': [{'answers': [{'answer_start': 28, 'text': 'лихорадкой'}],
   'id': '12786',
   'question': 'Чем заболел Байрон в Миссолонги?'}]}

In [ ]:
dev_data[0]['qas'][0]['question'] + ' ' + dev_data[0]['qas'][0]['answers'][0]['text']

'Чем заболел Байрон в Миссолонги? лихорадкой'

In [ ]:
69682

69682

In [ ]:
t_t = tokenizer.encode_plus('answer',
                        add_special_tokens=True,
                        padding='max_length',
                        max_length=50,
                        truncation=True,
                        return_tensors="pt")
t_t

{'input_ids': tensor([[20954,     1,     0,     0,     0,     0,     0,     0,     0,     0,
             0,     0,     0,     0,     0,     0,     0,     0,     0,     0,
             0,     0,     0,     0,     0,     0,     0,     0,     0,     0,
             0,     0,     0,     0,     0,     0,     0,     0,     0,     0,
             0,     0,     0,     0,     0,     0,     0,     0,     0,     0]]), 'attention_mask': tensor([[1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
         0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
         0, 0]])}

In [ ]:
x = [dev_data[0]['qas'][0]['question'], dev_data[0]['qas'][0]['answers'][0]['text']]
t_t = tokenizer.encode_plus(x,
                        add_special_tokens=True,
                        padding='max_length',
                        max_length=50,
                        truncation=True,
                        return_tensors="pt")
t_t

{'input_ids': tensor([[2, 2, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
         0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
         0, 0]]), 'attention_mask': tensor([[1, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
         0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
         0, 0]])}

In [5]:
class QGDataset:
    def __init__(self, dataset: List[dict], tokenizer) -> None:
        self.dataset = dataset
        self.tokenizer_context = lambda x: tokenizer.encode_plus(x,
                                                         add_special_tokens=True,
                                                         padding='max_length',
                                                         max_length=350,
                                                         truncation=True,
                                                         return_tensors="pt")
        self.tokenizer_question_answer = lambda x: tokenizer.encode_plus(x,
                                                         add_special_tokens=True,
                                                         padding='max_length',
                                                         max_length=200,
                                                         truncation=True,
                                                         return_tensors="pt")

    def __len__(self) -> int:
        return len(self.dataset)

    def __getitem__(self, idx) -> dict:
        context = self.dataset[idx]['context'][0:150]
        question = self.dataset[idx]['qas'][0]['question'][0:100]
        answer = self.dataset[idx]['qas'][0]['answers'][0]['text']
        qa_text = 'question' + ' ' + question + ' ' + 'answer' + ' ' + answer
        encodings_context = self.tokenizer_context(context)
        encoding_question_answering = self.tokenizer_question_answer(qa_text)

        result = {
            'input_ids': encodings_context['input_ids'].flatten(),
            'target_ids': encoding_question_answering['input_ids'].flatten(),
            'input_attention_mask': encodings_context['attention_mask'].flatten(),
            'target_attention_mask': encoding_question_answering['attention_mask'].flatten(),
        }
        # print(result)
        return result

        

In [6]:
def collate_fn(examples):
    return tokenizer.pad(examples, return_tensors='pt')

train_dataset = QGDataset(train_data, tokenizer)
train_iter = DataLoader(dataset=QGDataset(train_data, tokenizer),
                        batch_size=4)
dev_iter = DataLoader(dataset=QGDataset(dev_data, tokenizer),
                        batch_size=8)

In [7]:
for _ in train_iter:
    pass

In [ ]:
next(iter(train_iter))

# Model

In [8]:
import time


class QGmT5model(pl.LightningModule):
    def __init__(self, lr):
        super().__init__()
        self.lr = lr
        self.model = MT5ForConditionalGeneration.from_pretrained('google/mt5-small')

        self.tokenizer = T5Tokenizer.from_pretrained('google/mt5-small')
        self.rouge = load_metric('rouge')

        self.save_hyperparameters()

    def parse_score(self, result):
        return {k: round(v.mid.fmeasure * 100, 4) for k, v in result.items()}
    
    def forward(self, input_ids, input_attention_mask, 
                target_attention_mask=None, target_ids=None,):
        
        result = self.model(
            input_ids=input_ids,
            attention_mask=input_attention_mask,
            labels=target_ids)
        
        return result
    
    def lmap(self, f, x):
        return list(map(f, x))
    

    def _step(self, batch):
        lm_labels = batch['target_ids']
        lm_labels[lm_labels[:, :] == self.tokenizer.pad_token_id] = -100

        # print(f'input_ids: {batch["input_ids"].shape}')
        # print(f'input_attention_mask: {batch["input_attention_mask"].shape}')

        # print(f'target_idx: {batch["target_ids"].shape}')
        # print(f'target_attention_mask: {batch["target_attention_mask"].shape}')

        outputs = self(
            input_ids=batch["input_ids"],
            input_attention_mask=batch["input_attention_mask"],
            target_ids=batch["target_ids"],
            target_attention_mask=batch['target_attention_mask']
        )
    
        loss = outputs[0]
        return loss
    
    def ids_to_clean_text(self, generated_ids):
        gen_text = self.tokenizer.batch_decode(
            generated_ids, skip_special_tokens=True, clean_up_tokenization_spaces=True
        )
        return self.lmap(str.strip, gen_text)
    
    def _generative_step(self, batch) :
        
        t0 = time.time()
        
        generated_ids = self.model.generate(
            batch["input_ids"],
            attention_mask=batch["input_attention_mask"],
            use_cache=True,
            decoder_attention_mask=batch['target_attention_mask'],
            max_length=100, 
            num_beams=2,
            repetition_penalty=2.5, 
            length_penalty=1.0, 
            early_stopping=True
        )
        preds = self.ids_to_clean_text(generated_ids)
        target = self.ids_to_clean_text(batch["target_ids"])
            
        gen_time = (time.time() - t0) / batch["input_ids"].shape[0]  
    
        loss = self._step(batch)
        base_metrics = {'val_loss': loss}
        self.log_dict({'val_loss': loss})

        summ_len = np.mean(self.lmap(len, generated_ids))
        base_metrics.update(gen_time=gen_time, gen_len=summ_len, preds=preds, target=target)
        print(preds)
        print(target)
        self.rouge.add_batch(predictions=preds, references=target)
        
#         rouge_results = self.rouge_metric.compute() 
#         rouge_dict = self.parse_score(rouge_results)
#         base_metrics.update(rouge1=rouge_dict['rouge1'], rougeL=rouge_dict['rougeL'])
        
        return base_metrics
    
    def training_step(self, batch, batch_idx):
        loss = self._step(batch)

        logs = {"train_loss": loss}

        self.log_dict(logs)

        return loss
    
    def validation_step(self, batch, batch_idx):
        return self._generative_step(batch)
    
    def validation_epoch_end(self, outputs):
        
        avg_loss = torch.stack([x["val_loss"] for x in outputs]).mean()
        logs = {"val_loss": avg_loss}
        
        rouge_results = self.rouge.compute() 
        rouge_dict = self.parse_score(rouge_results)
    
        logs.update(rouge1=rouge_dict['rouge1'], rougeL=rouge_dict['rougeL'])

        self.log_dict(logs)
        
        ## Clear out the lists for next epoch
        self.target_gen= []
        self.prediction_gen=[]
        return {"avg_val_loss": avg_loss, 
                "rouge1" : rouge_results['rouge1'],
                "rougeL" : rouge_results['rougeL']}
    
    def configure_optimizers(self):
        optimizer = AdamW(self.model.parameters(), lr=self.lr, eps=1e-8)
        return optimizer

In [ ]:
comet_logger = CometLogger(
    api_key="HWfJT3eyByVJWe4nEbi1pGosA",
    workspace="danildmitriev1999",
    project_name="qa",
    experiment_name="mT5 QG",
)

CometLogger will be initialized in online mode


In [9]:
lr = 3e-4

N_EPOCHS = 1
CLIP = 1.5

model = QGmT5model(lr).to(device)

trainer = Trainer(max_epochs=N_EPOCHS,
                  gpus=1,
                gradient_clip_val=CLIP,
                progress_bar_refresh_rate=1,
                log_every_n_steps=3,
                )

GPU available: True, used: True
TPU available: False, using: 0 TPU cores


In [10]:
trainer.fit(model, train_iter, dev_iter)

LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]

  | Name  | Type                        | Params
------------------------------------------------------
0 | model | MT5ForConditionalGeneration | 300 M 
------------------------------------------------------
300 M     Trainable params
0         Non-trainable params
300 M     Total params
1,200.707 Total estimated model params size (MB)


['<extra_id_0>. <extra_id_10>', '<extra_id_0>.', '<extra_id_0>.', '<extra_id_0>.) <extra_id_37> с двигателем внутреннего сгорания', '<extra_id_0>.', '<extra_id_0>. <extra_id_10>. <extra_id_10>', '<extra_id_0>.', '<extra_id_0> Швейцарии']
['question Чем заболел Байрон в Миссолонги? answer лихорадкой', 'question Как отводятся излишки тепла у млекопитающих? answer потоотделением', 'question Что нарушают хромосомные аберрации? answer Структуру хромосом.', 'question В каком городе первый в мире городской автобус с двигателем внутреннего сгорания вышел на маршрут? answer в Лондоне', 'question В каком режиме проходят подлимитные операции? answer в режиме офлайн', 'question Находит ли отражение в алфавите фонетическая связь слогов? answer не находит', 'question Каким стал стиль последних работ художника? answer нервным и гнетущим', 'question Сколько человек родилось в 2008 году у граждан Швейцарии? answer 1147 человек']
['<extra_id_0>.', '<extra_id_0>.', '<extra_id_0>.', '<extra_id_0>.', '<ext

Global seed set to 294


In [ ]:
# from comet_ml import Experiment
# name = 't5_QG'
# trainer.save_checkpoint(f"/content/save_models/{name}.ckpt")
# experiment = Experiment(
#     api_key='HWfJT3eyByVJWe4nEbi1pGosA', project_name='qa',workspace='danildmitriev1999')

# experiment.log_model("t5_QG", f"/content/save_models/{name}.ckpt")

In [242]:
def generate_question(context, tokenizer, model):
    tokens = tokenizer.encode_plus(context,
                                    add_special_tokens=True,
                                    padding='max_length',
                                    max_length=350,
                                    truncation=True,
                                    return_tensors="pt")
    with torch.no_grad():
        generated_ids = model.generate(
                tokens["input_ids"],
                attention_mask=tokens["attention_mask"],
                use_cache=True,
                max_length=100, 
                num_beams=25,
                repetition_penalty=2.5, 
                length_penalty=1.5, 
                early_stopping=True
            )
    
    pred = [tokenizer.decode(generated_id, skip_special_tokens=True, clean_up_tokenization_spaces=True) 
    for generated_id in generated_ids]

    return pred

In [12]:
tr_model = model.model

In [160]:
dev_data[25]['context']

'Термин кратер принят в планетной номенклатуре — единой системе, однозначно идентифицирующей детали рельефа на поверхности Луны, что позволяет легко опознать и описать эти структуры. Присвоением официального названия, с момента своего основания в 1919 году, занимается Международный астрономический союз (МАС). Несмотря на то что за 360 лет истории лунной номенклатуры были присвоены имена не укладывающиеся в принятую схему, Международным астрономическим союзом приняты следующие правила. Кратеры на Луне, как правило, получают своё название в честь умерших выдающихся учёных, инженеров и исследователей, внёсших значительный, фундаментальный вклад в своей области. Кроме того, кратеры вокруг Моря Москвы названы в честь погибших советских космонавтов, а кратеры вокруг кратера Аполлон названы в честь погибших американских астронавтов. Это правило может быть распространено и для других космических держав, которые потеряют своих космонавтов. Для маленьких кратеров используются только имена, без ф

In [161]:
generate_question(dev_data[25]['context'], tokenizer, tr_model)

['question Где находится кратер? answer В честь умерших']

In [277]:
custom_context = "Происхождение Александра Сергеевича Пушкина идёт от разветвлённого нетитулованного дворянского рода Пушкиных, восходившего по генеалогической легенде к «мужу честну» Ратше[6][~ 3]. Пушкин неоднократно писал о своей родословной в стихах и прозе; он видел в своих предках образец истинной «аристократии», древнего рода, честно служившего отечеству, но не снискавшего благосклонности правителей и «гонимого». Не раз он обращался (в том числе в художественной форме) и к образу своего прадеда по матери — африканца Абрама Петровича Ганнибала, ставшего слугой и воспитанником Петра I, а потом военным инженером и генералом. В XVII веке предки Пушкина по отцу не поднимались выше придворного чина стольника. Прадед, живший в эпоху Петра I, Александр Петрович Пушкин, был сержантом гвардии и в 1725 году в припадке безумия убил свою жену; дед, Лев Александрович, был полковником артиллерии, капитаном гвардии. Отец — Сергей Львович Пушкин (1770—1848), светский острослов и поэт-любитель. Мать Пушкина — Надежда Осиповна (1775—1836), внучка Ганнибала. Дядя по отцу, Василий Львович (1766—1830), был известным поэтом круга Карамзина. Из детей Сергея Львовича и Надежды Осиповны, кроме Александра, выжили дочь Ольга (в замужестве Павлищева, 1797—1868) и сын Лев (1805—1852)"

custom_context

'Происхождение Александра Сергеевича Пушкина идёт от разветвлённого нетитулованного дворянского рода Пушкиных, восходившего по генеалогической легенде к «мужу честну» Ратше[6][~ 3]. Пушкин неоднократно писал о своей родословной в стихах и прозе; он видел в своих предках образец истинной «аристократии», древнего рода, честно служившего отечеству, но не снискавшего благосклонности правителей и «гонимого». Не раз он обращался (в том числе в художественной форме) и к образу своего прадеда по матери — африканца Абрама Петровича Ганнибала, ставшего слугой и воспитанником Петра I, а потом военным инженером и генералом. В XVII веке предки Пушкина по отцу не поднимались выше придворного чина стольника. Прадед, живший в эпоху Петра I, Александр Петрович Пушкин, был сержантом гвардии и в 1725 году в припадке безумия убил свою жену; дед, Лев Александрович, был полковником артиллерии, капитаном гвардии. Отец — Сергей Львович Пушкин (1770—1848), светский острослов и поэт-любитель. Мать Пушкина — Над

In [279]:
for i in range(5):
    print(generate_question(custom_context, tokenizer, tr_model))

['question Когда Пушкин был сержантом и генералом? answer В 1775 году']
['question В каком веке Пушкин был сержантом? answer в художественной традиции']
['question В каком году поэт-любитель? answer В XVII веке.']
['question В каком году встретил Пушкин? answer в XVIII веке.']
['question В каком году Пушкин посвятил свою родство? answer в 1775 году']
